In [1]:

cv::Mat kernel = cv::getGaborKernel(cv::Size(kernel_size,kernel_size), sig, th, lm, gm, ps);

SyntaxError: invalid syntax (<ipython-input-1-c5493d5a18ea>, line 1)

In [ ]:
cv::filter2D(src_f, dest, CV_32F, kernel);

In [2]:
Mat in = imread("/Users/charliethebear/Documents/Lab/Entropy/Raw/superficial.tif",0);          // load grayscale
Mat dest;
Mat src_f;
in.convertTo(src_f,CV_32F);

int kernel_size = 31;
double sig = 1, th = 0, lm = 1.0, gm = 0.02, ps = 0;
cv::Mat kernel = cv::getGaborKernel(cv::Size(kernel_size,kernel_size), sig, th, lm, gm, ps);
cv::filter2D(src_f, dest, CV_32F, kernel);

cerr << dest(Rect(30,30,10,10)) << endl; // peek into the data

Mat viz;
dest.convertTo(viz,CV_8U,1.0/255.0);     // move to proper[0..255] range to show it
imshow("k",kernel);
imshow("d",viz);
waitKey();

SyntaxError: invalid syntax (<ipython-input-2-83238591f6bf>, line 1)

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Grayscale
def BGR2GRAY(img):
    # Grayscale
    gray = 0.2126 * img[..., 2] + 0.7152 * img[..., 1] + 0.0722 * img[..., 0]
    return gray

# Gabor
def Gabor_filter(K_size=111, Sigma=10, Gamma=1.2, Lambda=10, Psi=0, angle=0):
    # get half size
    d = K_size // 2

    # prepare kernel
    gabor = np.zeros((K_size, K_size), dtype=np.float32)

    # each value
    for y in range(K_size):
        for x in range(K_size):
            # distance from center
            px = x - d
            py = y - d

            # degree -> radian
            theta = angle / 180. * np.pi

            # get kernel x
            _x = np.cos(theta) * px + np.sin(theta) * py

            # get kernel y
            _y = -np.sin(theta) * px + np.cos(theta) * py

            # fill kernel
            gabor[y, x] = np.exp(-(_x**2 + Gamma**2 * _y**2) / (2 * Sigma**2)) * np.cos(2*np.pi*_x/Lambda + Psi)

    # kernel normalization
    gabor /= np.sum(np.abs(gabor))

    return gabor


def Gabor_filtering(gray, K_size=111, Sigma=10, Gamma=1.2, Lambda=10, Psi=0, angle=0):
    # get shape
    H, W = gray.shape

    # padding
    gray = np.pad(gray, (K_size//2, K_size//2), 'edge')

    # prepare out image
    out = np.zeros((H, W), dtype=np.float32)

    # get gabor filter
    gabor = Gabor_filter(K_size=K_size, Sigma=Sigma, Gamma=Gamma, Lambda=Lambda, Psi=0, angle=angle)

    # filtering
    for y in range(H):
        for x in range(W):
            out[y, x] = np.sum(gray[y : y + K_size, x : x + K_size] * gabor)

    out = np.clip(out, 0, 255)
    out = out.astype(np.uint8)

    return out

def Gabor_process(img):
    # get shape
    H, W, _ = img.shape

    # gray scale
    gray = BGR2GRAY(img).astype(np.float32)

    # define angle
    As = [0, 45, 90, 135]

    # prepare pyplot
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0, hspace=0, wspace=0.2)

    out = np.zeros([H, W], dtype=np.float32)

    # each angle
    for i, A in enumerate(As):
        # gabor filtering
        _out = Gabor_filtering(gray, K_size=7, Sigma=1.5, Gamma=1.2, Lambda=6, angle=A)
        # add gabor filtered image
        out += _out

    # scale normalization
    out = out / out.max() * 255
    #out = out.astype(np.uint8)

    return out


# Read image
img = cv2.imread('/home/charlie/projects/entropy/img/Raw_superficial/reactive/active/6_1054482-8_20170926_OCT_00000256.bmp_superficial.jpg')

# gabor process
out = Gabor_process(img)

#out_binarized = cv2.threshold(out,0,255,cv2.THRESH_OTSU)


cv2.imwrite("/home/charlie/projects/entropy/img/test/gabor.jpg", out)
#cv2.imwrite("bin.jpg", out_binarized)
#cv2.imshow("image", img2)
#cv2.imshow("result", out)
#cv2.imshow("binarized", out_binarized)
cv2.waitKey(0)

-1

<Figure size 640x480 with 0 Axes>

In [ ]:
gray = cv2.cvtColor(out, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 75, 255, cv2.THRESH_BINARY)[1]
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

cnts = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    area = cv2.contourArea(c)
    if area < 15:
        cv2.drawContours(opening, [c], -1, (0,0,0), -1)

cv2.imshow('thresh', thresh)
cv2.imshow('opening', opening)
cv2.waitKey(0)

In [ ]:
gray = cv2.cvtColor(out, cv2.COLOR_BGR2GRAY)
thresh = cv2.threshold(gray, 75, 255, cv2.THRESH_BINARY)[1]
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3))
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)

cnts = cv2.findContours(opening, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cnts[0] if len(cnts) == 2 else cnts[1]
for c in cnts:
    area = cv2.contourArea(c)
    if area < 15:
        cv2.drawContours(opening, [c], -1, (0,0,0), -1)

cv2.imshow('thresh', thresh)
cv2.imshow('opening', opening)
cv2.waitKey(0)